In [1]:
# Super temporary notebook for debugging purposes. 

import main

main.main()

# Seems like an unusual shape. 
# Ah, the output has to be the inverse.
# For reference, input zyx tile size is: 
# (2500, 2304, 2304)
# In absolute coordinates: 
# (2500 * 0.8, 2304 * 0.3, 2304 * 0.3) = (2000, 691, 691)
# Deskewed: 
# (2000 * 2, 691, 691) = (4000, 691, 691)
# Inverse Sampled: 
# (4000 * 2, 691 * 2, 691 * 2) = (8000, 1382, 1382)
# For one tile. 
# This makes sense-- deskewing/oversampling in z and undersampling in y. 

2023-10-23 03:59 Number of Tiles: 4
2023-10-23 03:59 OUTPUT_VOLUME_SIZE=(5590, 12000, 1606)
2023-10-23 03:59 Estimated Total Cells: 576
2023-10-23 03:59 cell_box=array([   0, 1024,    0,  512,    0,  512])
2023-10-23 03:59 overlapping_tiles=[3]
2023-10-23 04:00 Cell 0/576: 12.20632553100586
2023-10-23 04:00 cell_box=array([   0, 1024,    0,  512,  512, 1024])
2023-10-23 04:00 overlapping_tiles=[3]
2023-10-23 04:00 AABB of Crop belonging to image 3: 0, 271, 0, 442, 702, 1618
/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
2023-10-23 04:00 Cell 1/576: 15.74216604232788
2023-10-23 04:00 cell_box=array([   0, 1024,    0,  512, 1024, 1536])
2023-10-23 04:00 overlapping_tiles=[3]
2023-10-23 04:00 AABB of Crop belonging to image 3: 0, 58

In [ ]:
# Appears to be running. 
# Now to check accuracy, will bring in the plots. 

# Suspicious b/c not getting the warning that grid_sample is actually running. 
# Or perhaps I only get the warning on clipping cases. 



In [4]:
# Double check output validility: 
import matplotlib.pyplot as plt
import zarr
import dask.array as da

in_path = "/root/capsule/scratch/tmp_output.zarr/0"
arr = zarr.open(in_path, mode='r')
arr = da.from_zarr(arr)

print(arr.shape)

plt.imshow(arr[0, 0, :, 500, :])
plt.savefig('/scratch/zx.jpg')

# plt.imshow(arr[0, 0, :, :5000, 0])
# plt.savefig('/scratch/zy.jpg')

# Appears to be doing something correct, but not sure. 
# need to load in other dataset + add deskewing transform. 
# Surefire way to debug: introduce plots again-- go back in capsule commit history. 

# Plan: 
# Load Polina dataset + hardcoded deskew transform
# Run codebase refactor
# See if things are correct.
# If not correct, bring in the plots. 
# Add resolution hardcoding, then add as input parameter. 
# Run on multi-cpu and multi-gpu. 
# Capture runtime metrics. 
# I estimate another 8 hours. 

# If things are correct, that's promising. 
# If not, you are in for a night. 


# (I'm just going to make the change, I have no patience for this)



(1, 1, 2565, 7617, 2398)


In [6]:
# Figuring out the input resolution access path

import xmltodict
from collections import OrderedDict

xml_path = "/root/capsule/data/Christian-ME-New-Lasers_stitch_output/bigstitcher_Christian-ME-New-Lasers.xml"

with open(xml_path, "r") as file:
    data: OrderedDict = xmltodict.parse(file.read())

resolution_str = data["SpimData"]["SequenceDescription"]["ViewSetups"]["ViewSetup"][0]['voxelSize']['size']
resolution_xyz = resolution_str.split(" ")
tuple(resolution_xyz[::-1])



('0.8', '0.298', '0.298')

In [13]:
import yaml
import numpy as np

yaml_path = "./config.yaml"

with open(yaml_path, 'r') as f:
    yaml_dict = yaml.safe_load(f)

# Indeed, this is a list of lists. 
# You need to format these as transforms. 
tfm_list = yaml_dict['algorithm_parameters']["post_registration_transforms_zyx"]

np.array(tfm_list[0])




array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [1]:
t = [1, 2, 3, 4]

[0, *t, 5] 
# Works. 

[0, 1, 2, 3, 4, 5]